In [72]:
%load_ext autoreload
%autoreload 2

from transformers import CLIPProcessor, CLIPModel, CLIPConfig, VisionTextDualEncoderModel, VisionTextDualEncoderProcessor, AutoTokenizer,AutoImageProcessor
from dataset import CLIPDataset, make_dataset_clip
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
from transformers import DefaultDataCollator


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
config = CLIPConfig.from_pretrained("openai/clip-vit-base-patch32")

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

image_processor = AutoImageProcessor.from_pretrained("openai/clip-vit-base-patch32")

# model = VisionTextDualEncoderModel.from_vision_text_pretrained(
#     "openai/clip-vit-base-patch32", "roberta-base"
# )

model = CLIPModel(config)

# processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


# dataset = CLIPDataset(processor)
dataset_train, dataset_val = make_dataset_clip(processor)



Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [74]:
data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir="clip_exp",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    # metric_for_best_model="accuracy",
    metric_for_best_model=None,
    push_to_hub=False,
    do_eval=False
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=None,
    
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [76]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,2.769500,No log
2,2.449600,No log
4,2.322000,No log
6,2.012700,No log
8,1.670200,No log
10,1.366000,No log
12,0.992900,No log
14,0.702700,No log
16,0.530400,No log
18,0.336300,No log


/home/zjf/miniconda3/envs/nlp/lib/python3.9/site-packages/transformers/models/clip/processing_clip.py:143: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/home/zjf/miniconda3/envs/nlp/lib/python3.9/site-packages/transformers/models/clip/processing_clip.py:143: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/home/zjf/miniconda3/envs/nlp/lib/python3.9/site-packages/transformers/models/clip/processing_clip.py:143: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/home/zjf/miniconda3/envs/nlp/lib/python3.9/site-packages/transformers/models/clip/processing_clip.py:143: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/home/zjf/miniconda3/envs/nlp/lib/python3.9/site-packages/transformers/model

TrainOutput(global_step=570, training_loss=1.0561671821694625, metrics={'train_runtime': 1120.7132, 'train_samples_per_second': 33.14, 'train_steps_per_second': 0.509, 'total_flos': 136692445220100.0, 'train_loss': 1.0561671821694625, 'epoch': 29.23})

In [ ]:
trainer.save_model("/home/zjf/repos/proj_595/simple_exp/my_saved_model.pth")

In [77]:
from torch.utils.data import DataLoader
import torch

In [ ]:
test_dataloader = DataLoader(dataset_val, batch_size=64, shuffle=False)
for data in test_dataloader:
    print(data)

In [ ]:
dataset_val[0]

new_input = {}
random_id = torch.tensor([49405, 5435,  1704, 49407, 49407]).cuda()
new_input['input_ids'] = torch.stack([dataset_val[0]['input_ids'].cuda(), random_id]).cuda()
new_input['pixel_values'] = dataset_val[0]['pixel_values'].unsqueeze(0).cuda()

In [ ]:
new_input['pixel_values'].shape

In [ ]:
outputs = model(**new_input)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

In [ ]:
probs.argmax()

In [78]:
test_dataloader = DataLoader(dataset_val, batch_size=64, shuffle=False)

id_list = []
for data in test_dataloader:
    id_list.append(data['input_ids'])



In [88]:
all_ids = torch.unique(torch.cat(id_list), dim=0)

In [89]:
from tqdm import tqdm

In [93]:
eval_dataloader = DataLoader(dataset_val, batch_size=1, shuffle=False)

all_pred = 0
correct_pred = 0

for data in tqdm(eval_dataloader):
    new_input = {}
    new_input['input_ids'] = all_ids
    new_input['pixel_values'] = data['pixel_values'].cuda()
    outputs = model(**new_input)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
    correct_num = find_idx(all_ids, data['input_ids'].cuda())
    print('pred:', probs.argmax())
    # break
    print('correct: ', correct_num)
    if probs.argmax() == correct_num:
        correct_pred += 1
    all_pred += 1


  0%|          | 0/309 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
acc = correct_pred / all_pred

In [ ]:
acc

In [ ]:
torch.where(all_ids == data['input_ids'].cuda().squeeze())
# print(data['input_ids'].shape)

def find_idx(all_ids, query):
    for i in range(len(all_ids)):
        if (query.cuda() == all_ids[i].cuda().squeeze()).all():
            return i

print(find_idx(all_ids, data['input_ids']))